In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

class GridWorld:
    def __init__(self, size, start, end, obstacles):
        self.size = size
        self.start = start
        self.end = end
        self.obstacles = obstacles
        self.grid = np.zeros(size)
        self.grid[end] = 1
        for obs in obstacles:
            self.grid[obs] = -1

    def is_terminal(self, state):
        return state == self.end

    def get_next_state(self, state, action):
        next_state = list(state)
        if action == 0: # up
            next_state[0] = max(0, state[0]-1)
        elif action == 1: # down
            next_state[0] = min(self.size[0]-1, state[0]+1)
        elif action == 2: # left
            next_state[1] = max(0, state[1]-1)
        elif action == 3: # right
            next_state[1] = min(self.size[1]-1, state[1]+1)
        
        if tuple(next_state) in self.obstacles:
            next_state = state

        return tuple(next_state)

    def get_reward(self, state, action, next_state):
        if next_state == self.end:
            return 1
        else:
            return -0.01

size = (16, 16)
start = (0, 0)
end = (15, 15)
obstacles = [(3, 3), (3, 4), (3, 5), (4, 5), (5, 5), (5, 4), (5, 3)]

env = GridWorld(size, start, end, obstacles)

def epsilon_greedy_policy(Q, state, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, 3)
    else:
        return np.argmax(Q[state])

def first_visit_mc(env, episodes, alpha, gamma, epsilon, epsilon_decay):
    Q = np.zeros((size[0], size[1], 4))
    returns = {}

    for episode in range(episodes):
        state = env.start
        episode_states_actions = []
        episode_rewards = []

        while not env.is_terminal(state):
            action = epsilon_greedy_policy(Q, state, epsilon)
            next_state = env.get_next_state(state, action)
            reward = env.get_reward(state, action, next_state)

            episode_states_actions.append((state, action))
            episode_rewards.append(reward)
            state = next_state

        G = 0
        for t in reversed(range(len(episode_states_actions))):
            state, action = episode_states_actions[t]
            G = gamma * G + episode_rewards[t]

            if (state, action) not in episode_states_actions[:t]:
                if (state, action) not in returns:
                    returns[(state, action)] = []
                returns[(state, action)].append(G)
                Q[state][action] = Q[state][action] + alpha * (G - Q[state][action])

        epsilon *= epsilon_decay



In [2]:
def plot_gridworld(env, Q):
    grid = np.copy(env.grid)
    for i in range(size[0]):
        for j in range(size[1]):
            if grid[i, j] == 0:
                grid[i, j] = np.argmax(Q[(i, j)])
    plt.imshow(grid, cmap="cool")
    plt.colorbar(ticks=[-1, 0, 1, 2, 3])
    plt.clim(-1, 3)
    plt.xticks(range(size[1]), range(size[1]))
    plt.yticks(range(size[0]), range(size[0]))
    plt.gca().invert_yaxis()
    plt.show()

# Train the agent
episodes = 5000
alpha = 0.1
gamma = 0.99
epsilon = 0.3
epsilon_decay = 0.99

first_visit_mc(env, episodes, alpha, gamma, epsilon, epsilon_decay)

# Visualize the final policy
plot_gridworld(env, Q)


KeyboardInterrupt: 